# Generating the Flextegrity Lattice

Our purpose here is to elaborate a classic Python framework for storing and writing geometric information in a variety of formats, suitable for ray tracing and use within CAD programs.

The target architecture we want to model is [Flextegrity](http://c6xty.com), a tension-compression lattice based on holding compression islands in relative positions, in an all-way space.

### CCP (Cubic Close Packing)
The general case CCP lattice will form our background.  The cubic closest packed matrix of balls serves as our "holodeck" (our substitute XYZ grid), for "tuning in" our Flextegrity lattices.

Where spheres touch, they define the diamond faces of a rhombic dodecahedron, each tangent to twelve neighbors.  This marks the "high water mark" for sphere size (inter-tangency) and is the CCP itself.

As spheres begin to contract, without losing their relative positions, we're in a position to study alternative adjoining or holding strategies.  

How might these spheres still connect to each other in a way that preserves their intitial positions, and yet may allow from abberrational distortions, i.e. flexibility?  Our lattice is not brittle.

### CCP in Architecture

The center-to-center strategy is well known to some architects as a space frame patented as the "octet truss" by R. Buckminster Fuller, and featured as "kites" in earlier studies by Alexander Graham Bell.  

NASA went with the octet truss as a basic space frame for orbiting space station designs.

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/3604089857/in/photolist-6utUmi-gzmEy1-ekFr83-gzmEhu-5Pskff-5Ff3Hc-5zM26h-7kM3FK-9ho3VF-SihXUN-3Cdjdw-vaBoTv-AUNWd6-r8nXpZ-r8o2un-vTMVDX-vTMXoD-9hrbAY-pnSHJt-pEkQ8L-pEkQb1-pnRKqt-HMv9ay-DFodCy-5bVG87-5bVGd7-6ejsoR-5bRq6a-5bVGj5-HMvahU-dZqij2-dZqjUp-fDCcB4-dZqk5g-dZw32d-dZw2e9-dZw23o-dZqiEr-dZw1HN-dZqjBR-dZqkqp-Sssbwf-S7nkQ3-4oQCpy-5rFY8v-n4cppG-5FjkKG-69L1bK-oBEGJt-oDqycH" title="Octet Truss"><img src="https://live.staticflickr.com/3326/3604089857_033f9f54c2.jpg" width="500" height="375" alt="Octet Truss"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

### Flextegrity

Sam Lanahan pioneered a three-way weaving of mutually orthogonal truss members, called struts in some models, that suggest different, possibly flexible, materials.  

Flexibility means countouring, as road pavement matches the contour of the underlying topography.

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/4249887521/in/photolist-7L5wXK-bFHyv8-8bdGfA-5xKsWB-6iQiQi-aBoaMG-9f5Bg7-6iUv5o-8YLgPj-7txMuM-7vYsSk-cwtfnN-9Y6Jq5-VBwCaX-7Wkd9P-7vYsE6-6CyK5a-7YNVtZ-83biTg-7FqRR3-7NGzHS-7QW944-7wzTcq-6tQosA-5QB1z4-7YS7Uo-8pNRGs-6h9qoA-pkPux3-6CCTpC-7r71wq-9WvZsv-7nJjtA-91Zd3u-68oXAX-91W6pn-7KvFwT-7Pz6gD-8oTV56-8UWce6-7txMip-8iYwQE-crzsp1-7txPja-9Y8Bvq-zCJpe6-Hie7g4-8ryyt2-P6io19-dPU3pc" title="Flextegrity Model"><img src="https://live.staticflickr.com/2650/4249887521_9525392db4_z.jpg" width="640" height="480" alt="Flextegrity Model"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

### Cubic Reference Cells

Our cubic reference cell, surrounding any compression island, is defined by the short diagonals of the rhombic dodecahedron's diamond faces.  

The cube and dodeca have relative volumes 3 and 6 (1 and 2).  Each dodeca carves out 1/6th of the six face-adhering neighboring reference cubes, pictured as void (no balls therein). This effectively describes the rhombic dodecahedron: a reference cube with six sixths added (so double the cube's volume).

The rhombic dodecahedron is the space-filler associated with the CCP, which each ball nested snugly inside one.  Relative the tetrahedron of four CCP balls, intertangent, the rhombic dodecahedron is six tetravolumes.

From the Fuller Archives:

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/4147716381/in/photolist-7jw8zp-27m279d-28ryrb8-27m2SHJ-sb9rRn-274R4SM-274RBWX-28rzEyt-27aZfmh-28nawPU-JMAMGp-2hGyuQ1-6Lm3DN-sutG6M-8rBHYW-WgfUsF-6XSpXY-2ypHhr-W18AtK-7DSum6-9dqru5-9dBJbo-XvSatr-ssnvYM-saV86K-Am4sCQ-27m33iL-Vrm16F-rxt3hQ-2g9hw2G-HUjd6Y-qvSspT-suiyJu-pyYUi2-qvJVYB-WBfCVQ-K87vgR-JjTpKz-HoQ4xE-HoPrdA-surYzt-pySJoq-91hkkf-HUjyB7-sb9AwP-ssnxz2-saLYqJ-b4FXeM-rvyvsT-sskHhP" title="Legendary Concentric Hierarchy"><img src="https://live.staticflickr.com/2573/4147716381_7991c273b2.jpg" width="389" height="500" alt="Legendary Concentric Hierarchy"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Consider an icosahedron with its three XYZ golden rectangles parallel to those of our reference cube.  If the rectangles terminate in the cube's faces, then connecting bands to adjacent cube edges will provide a mechanism for inter-icosahedron joining.

![](flextegrity.png)

As the icosahedrons grow such that their phi rectangles protrude through the cube's faces, into the neighboring cube domains, all of which are void (empty of sphere centers), the bands to the reference cube edges will slant, like the sides of a tent from a ridge pole, but not as steeply. This slanting band will meet another at 180 degrees, coming from a neighboring icosahedron.  Twelve icosahedron's around a nuclear icosahedron, will be held in place in this way.

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/3484407906/in/photolist-7L5wXK-bFHyv8-8bdGfA-5xKsWB-6iQiQi-aBoaMG-9f5Bg7-6iUv5o-8YLgPj-7txMuM-7vYsSk-cwtfnN-9Y6Jq5-VBwCaX-7Wkd9P-7vYsE6-6CyK5a-7YNVtZ-83biTg-7FqRR3-7NGzHS-7QW944-7wzTcq-6tQosA-5QB1z4-7YS7Uo-8pNRGs-6h9qoA-pkPux3-6CCTpC-7r71wq-9WvZsv-7nJjtA-91Zd3u-68oXAX-91W6pn-7KvFwT-7Pz6gD-8oTV56-8UWce6-7txMip-8iYwQE-crzsp1-7txPja-9Y8Bvq-zCJpe6-Hie7g4-8ryyt2-P6io19-dPU3pc/" title="Flextegrity Design"><img src="https://live.staticflickr.com/3539/3484407906_47eec921c5_z.jpg" width="640" height="480" alt="Flextegrity Design"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>


## Python Framework

The Python framework we're developing has a generic purpose, of introducing simple vectorial geometry in conjunction with the Concentric Hierarchy well-known to School of Tomorrow students.  The content is in the form of a Jupyter Notebook on Github.

The skeletal Polyhedron below is meant to be subclassed and further fleshed out. This inherited piece provides mechanisms for resizing and translating the beast.  You may imagine adding rotation, as you learn more about vector-based computations based on technologies invented by Heaviside-Gibbs.

Python allows operator overloading such that multiplying a Polyhedron object by a scalar (\* triggers ```__mul__``` or ```__rmul__```) , say an int or float, will resize it in place.  A new Polyhedron is delivered however, following a more functional model with more immutability of state.  

Using the addition operator (+ triggers ```__add__``` or ```__radd__```), and supplying a vector (an arrow pointing to "where it should move"), the polyhedron may be translated (without resizing or rotating).  Again, the original polyhedron is unaffected, as only the face structure with labeled rings is essential topology.  

Acting on all the vertexes effectively scales and/or moves the object, but the Polyhedron so defined need not entangle itself with the original.

```python
>>> t = Tetrahedron()
>>> t2 = t + Qvector((2,1,1,0))
>>> t2.edges  # re-distilled
```

The strategy in both cases (resizing or moving) is similar:  the vertexes are what anchor the whole show, are the tent stakes as it were.  

To resize and move requires recomputing the vertexes, relative to an origin.  

Additionally, each subclass of Polyhedron needs to bring to the table its own dictionary of labeled faces.

Per the Cube:

```python
        # 6 faces
        self.faces = (('a','f','c','h'),('h','c','e','b'),
                      ('b','e','d','g'),('g','d','f','a'),
                      ('c','f','d','e'),('a','h','b','g'))
```

where these lowercase labels correspond to their uppercase global counterparts, the A-Z of the Concentric Hierarchy.

From the faces, a Polyhedron knows how to distill all the edge pairs.  The Edge is a type of object all on its own, consisting of only two vectors.  A vector, by definition, has its tail at the origin, and so only carries the information for one point in space (relative the fixed origin).  An Edge connects any two vectors, so a Polyhedron may be considered a network of such edges.

In [1]:
class Polyhedron:
    """
    Designed to be subclassed, not used directly
    """
    
    def scale(self, scalefactor):
        newverts = {}
        for v in self.vertexes:
            newverts[v] = self.vertexes[v] * scalefactor
        newme = type(self)()
        newme.vertexes = newverts
        newme.edges = newme._distill()
        return newme

    __mul__ = __rmul__ = scale

    def translate(self, vector):
        newverts = {}
        for v in self.vertexes:
            newverts[v] = self.vertexes[v] + vector
        newme = type(self)()
        newme.vertexes = newverts
        newme.edges = newme._distill()
        return newme

    __add__ = __radd__ = translate
    
    def _distill(self):

        edges = []
        unique = set()
        
        for f in self.faces:
            for pair in zip(f , f[1:] + (f[0],)):
                unique.add( tuple(sorted(pair)) )
        
        for edge in unique:
            edges.append( Edge(self.vertexes[edge[0]],
                               self.vertexes[edge[1]]) )

        return edges            

As provided, the Edge is little more than a two-vector storing device.  In various frameworks, it might be fleshed out to provide its own rendering methods.  By "rendering" I mean writing out in some other flavor of source code (outside of Python) such as in Scene Description Language (POV-Ray, free, open) or in Rhino (CAD).

Rather than customize the Edge here, the Scene Description Language rendering will be provided externally, as free-standing functions outside of Polyhedron or Edge types.

In [2]:
class Edge:

    """
    Edges are defined by two Vectors (above) and express as cylinder via draw().
    """

    def __init__(self, v0, v1):
        self.v0 = v0
        self.v1 = v1
             
    def __repr__(self):
        return 'Edge from %s to %s' % (self.v0, self.v1)

The master function we'll use for rendering a poly is ```draw_poly```.  It takes the poly in question as a first argument, then the filetype object to which the output will go.  Finally, some switches allow turning on and off rendering of specifically edges, vertexes and faces (V + F = E + 2).

In [3]:
def draw_vert(v): pass
def draw_face(f): pass
def draw_edge(e): pass

def draw_poly(p, the_file, v=True, f=True, e=True):
    
    ec = p.edge_color
    er = p.edge_radius
    vc = p.vert_color
    vr = p.vert_radius
    fc = p.face_color
    
    if v:
        for v in p.vertexes.values():
            draw_vert(v, vc, vr, the_file)

    if f:
        for f in p.faces:
            draw_face(f, fc, the_file)

    if e:
        for e in p.edges:
            draw_edge(e, ec, er, the_file)

In [4]:
from qrays import Qvector

![](qtet.png)

### Quadray Coordinates

For a change of pace, and to expand our horizons, the vector framework here introduced may seem unfamiliar.  Instead of the three positive and three negative unit vectors, spokes in a "jack" arrangement, we have just four basis vectors, scalable but not rotatable, and use linear combinations of those four to specify our point "out there" from the origin, which is (0,0,0,0).  Instead of a "jack" we're using a "caltrop".

![quadrays](https://upload.wikimedia.org/wikipedia/commons/9/99/Quadray.gif)

As discussed in [Generating the FCC](https://github.com/4dsolutions/School_of_Tomorrow/blob/master/Flextegrity_Lattice.ipynb), all twelve unique permutations of the quadray coordinates {2,1,1,0} give use the CCP positions relative the origin.  We may fill space with the CCP with integral addressing of rhombic dodecahedron centers.

### Globals: A-Z + Jitterbug

Perhaps controversially, the global namespace gets populated with a set of reference vectors, conveniently A-Z i.e. 26 ASCII characters (consecutive).  These are all vector sums of the original basis vectors, which go from the center of a tetrahedron in four directions, by convention to its points.

The edges of the Tetrahedron formed by the four basis vectors has the lengths we most care about and we measure in CCP sphere radii or diameters.  The Tetrahedron has edges = 1 CCP diameter (2 radii).

We get pretty much all of the Concentric Hierarchy vertexes with A-Z, but for the 12 vertexes of the icosahedron (from which its dual, and rhombic triacontahedron, may be derived).  The bridging to five-fold symmetry is accomplished by means of the Jitterbug Transformation in our curriculum.

In the computations below, the outer 12 positions are transformed into the three mutually orthogonal phi rectangles or "boards" of the icosahedron.

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [5]:
import math
PHI = (1 + math.sqrt(5))/2.0

A = Qvector((1,0,0,0))
B = Qvector((0,1,0,0))
C = Qvector((0,0,1,0))
D = Qvector((0,0,0,1))
E,F,G,H     = B+C+D, A+C+D, A+B+D, A+B+C
I,J,K,L,M,N = A+B, A+C, A+D, B+C, B+D, C+D
O,P,Q,R,S,T = I+J, I+K, I+L, I+M, N+J, N+K
U,V,W,X,Y,Z = N+L, N+M, J+L, L+M, M+K, K+J

# OPPOSITE DIAGONALS
# ZY WX
# RV OS
# TU PQ
control = (Z - T).length()

midface = (Z + Y)
gold    = 0.5 * PHI * midface/midface.length()
Zi = gold + J/J.length() * control/2
Yi = gold + M/M.length() * control/2

midface = (W + X)
gold    = 0.5 * PHI * midface/midface.length()
Wi = gold + J/J.length() * control/2
Xi = gold + M/M.length() * control/2

midface = (R + V)
gold    = 0.5 * PHI * midface/midface.length()
Ri = gold + I/I.length() * control/2
Vi = gold + N/N.length() * control/2

midface = (O + S)
gold    = 0.5 * PHI * midface/midface.length()
Oi = gold + I/I.length() * control/2
Si = gold + N/N.length() * control/2

midface = (T + U)
gold    = 0.5 * PHI * midface/midface.length()
Ti = gold + K/K.length() * control/2
Ui = gold + L/L.length() * control/2

midface = (P + Q)
gold    = 0.5 * PHI * midface/midface.length()
Pi = gold + K/K.length() * control/2
Qi = gold + L/L.length() * control/2

### Subclassing the Polyhedron Class
Subclassing may now commence.  At this level, customization, a kind of "rust" is tainting our hitherto Platonic framework with abberational details of a specific subculture, that of the POV-Ray users, the raytracers.

Secondary characteristics (as these were known to Descartes) include such not-Platonic features such as color, texture, even weight.  Tension and compression start to creep in, as real world properties.  All of this is for the better, but it complicates our code, so we have postponed, until now, any introduction of Scene Description Language.

In [6]:
class Tetrahedron(Polyhedron):
    
    def __init__(self):
        self.name = "Tetrahedron"
        self.edge_color = "rgb <{}, {}, {}>".format(1, 127/255, 80/255)
        self.edge_radius= 0.03
        self.vert_color = "rgb <{}, {}, {}>".format(1, 127/255, 80/255)
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"
        
        self.vertexes = dict(a = Qvector((1,0,0,0)),
                             b = Qvector((0,1,0,0)),
                             c = Qvector((0,0,1,0)),
                             d = Qvector((0,0,0,1)))
        
        # 4 faces
        self.faces = (('a','b','c'),('a','c','d'),
                      ('a','d','b'),('b','d','c'))

        self.edges = self._distill()
        
class Cube (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <0, 1, 0>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <1, 0, 0>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(a =  Qvector((1,0,0,0)),
                             b =  Qvector((0,1,0,0)),
                             c =  Qvector((0,0,1,0)),
                             d =  Qvector((0,0,0,1)),
                             e = -Qvector((1,0,0,0)),
                             f = -Qvector((0,1,0,0)),
                             g = -Qvector((0,0,1,0)),
                             h = -Qvector((0,0,0,1)))
        self.name = "Cube"
        
        # 6 faces
        self.faces = (('a','f','c','h'),('h','c','e','b'),
                      ('b','e','d','g'),('g','d','f','a'),
                      ('c','f','d','e'),('a','h','b','g'))

        self.edges = self._distill()

Future editions of this Jupyter Notebook might rely more on the pre-existing globals, A-Z, for getting these polyhedrons off the ground.  The code above goes back to the original Quadray coodinates of each vertex.

In the Cuboctahedron and Icosahedron, the code will marry itself to the global namespace, meaning these types have no stand-alone integrity outside the specific Notebook provided, or another similarly prepopulated.

Experimenting with translation:  create a new Tetrahedron object (canonically sized) and move it to the center of a neighboring CCP position, i.e. to the center of a tangent CCP ball relative the ball at the origin.

In [7]:
t = Tetrahedron()
t2 = t + Qvector((2,1,1,0))
t2.edges

[Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=2, b=2, c=1, d=0),
 Edge from ivm_vector(a=2, b=2, c=1, d=0) to ivm_vector(a=2, b=1, c=2, d=0),
 Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=2, b=1, c=2, d=0),
 Edge from ivm_vector(a=2, b=2, c=1, d=0) to ivm_vector(a=1, b=0, c=0, d=0),
 Edge from ivm_vector(a=3, b=1, c=1, d=0) to ivm_vector(a=1, b=0, c=0, d=0),
 Edge from ivm_vector(a=2, b=1, c=2, d=0) to ivm_vector(a=1, b=0, c=0, d=0)]

In [8]:
c = Cube()

## Scene Description Language (POV-Ray)

Since POV-Ray Scene Description Language is a whole world unto itself, much like VRML (Virtual Reality Markup Language), why not provide direct access to the header, so the end user, any reader of this Notebook, is empowered to play with lighting, camera position, available texturing libraries.

Our project is not about recreating the capable POV-Ray GUI in some alternative Python compatible GUI widget kit.  That could be fun to undertake, but we're just using povray at the command line, from within the bash shell (orginally on Mac OSX).  Something similar works on Windows, or on Linux (including on a Pi, where POV-Ray may be successfully compiled).

In [9]:
pov_header = \
"""
// Persistence of Vision Ray Tracer Scene Description File
// File: xyz.pov
// Vers: 3.6
// Desc: test file
// Date: Sat Sep  7 09:49:33 2019
// Auth: me
// ==== Standard POV-Ray Includes ====
#include "colors.inc"     // Standard Color definitions
#include "textures.inc"   // Standard Texture definitions
#include "functions.inc"  // internal functions usable in user defined functions

// ==== Additional Includes ====
// Don't have all of the following included at once, it'll cost memory and time
// to parse!
// --- general include files ---
#include "chars.inc"      // A complete library of character objects, by Ken Maeno
#include "skies.inc"      // Ready defined sky spheres
#include "stars.inc"      // Some star fields
#include "strings.inc"    // macros for generating and manipulating text strings

// --- textures ---
#include "finish.inc"     // Some basic finishes
#include "glass.inc"      // Glass textures/interiors
#include "golds.inc"      // Gold textures
#include "metals.inc"     // Metallic pigments, finishes, and textures
#include "stones.inc"     // Binding include-file for STONES1 and STONES2
#include "stones1.inc"    // Great stone-textures created by Mike Miller
#include "stones2.inc"    // More, done by Dan Farmer and Paul Novak
#include "woodmaps.inc"   // Basic wooden colormaps
#include "woods.inc"      // Great wooden textures created by Dan Farmer and Paul Novak

global_settings {assumed_gamma 1.0}
global_settings {ambient_light rgb<1, 1, 1> }

// perspective (default) camera
camera {
  location  <4, 0.1, 0.2>
  look_at   <0.0, 0.0,  0.0>
  right     x*image_width/image_height
}

// create a regular point light source
light_source {
  0*x                  // light's position (translated below)
  color rgb <1,1,1>    // light's color
  translate <-20, 15, 10>
}

// create a regular point light source
light_source {
  0*x                  // light's position (translated below)
  color rgb <1,1,1>    // light's color
  translate <20, -15, -10>
}

background { color rgb <1.0, 1.0, 1.0> }
"""

Finally, lets rewrite the functions left as stubs above.  Our ```draw_poly``` function cycles through all of these in concert.

As we're mainly concerned with polyhedrons in wireframe view (with "wires" as thick or thin as we set their radii), no work was expended on face rendering.

In [10]:
def draw_vert(v, c, r, t): 
    v = v.xyz()
    x,y,z = v.x, v.y, v.z
    data = "< %s, %s, %s >" % (x,y,z), r, c
    template = ("sphere { %s, %s texture "
                "{ pigment { color %s } } no_shadow }")
    print(template % data, file=t)
    
def draw_face(f, c, t): pass

def draw_edge(e, c, r, t):
    v = e.v0.xyz()
    v0 = "< %s, %s, %s >" % (v.x, v.y, v.z)
    v = e.v1.xyz()
    v1 = "< %s, %s, %s >" % (v.x, v.y, v.z)
    data = (v0, v1, r, c)
    template = ("cylinder { %s, %s, %s texture "
                        "{pigment { color %s } } no_shadow }")
    print(template % data, file=t)

In [11]:
with open("testme.pov", "w") as target:
    target.write(pov_header)
    draw_poly(c, target, f=False)
    draw_poly(t, target, f=False)

![](testme.png)

## APPENDIX:  Additional Source Code and Renderings

Below are those 12 quadrays to nearest CCP neighbors around the origin.  All 26 A-Z reference points are similarly integral (and positive in canonical form).

By means of translation, we may conjure a cube at the origin, clone it around, and put some other figure inside each, appropriately scaled.

A sphere in particular may be:

* tangent to the reference cube's six faces
* tangent to the reference cube's 12 mid-edges (high water mark)
* internal to the cube (shrinking)
* bulging from the cube (growing)

Now mentaly replace each sphere with a compression island, perhaps an icosahedron, and you're reading to weave some Flextegrity.

In [12]:
from itertools import permutations
g = permutations((2,1,1,0))
unique = {p for p in g}  # set comprehension
print(unique)

{(0, 1, 1, 2), (1, 0, 1, 2), (2, 0, 1, 1), (0, 2, 1, 1), (0, 1, 2, 1), (1, 2, 1, 0), (1, 1, 2, 0), (2, 1, 1, 0), (1, 0, 2, 1), (1, 2, 0, 1), (2, 1, 0, 1), (1, 1, 0, 2)}


In [13]:
def twelve_cubes():
    c = Cube()
    twelve_cubes = [ ]
    for v in unique:
        trans_vector = Qvector(v)
        twelve_cubes.append(c + trans_vector)
    return twelve_cubes

with open("multicube.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)

![](multicube.png)

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [14]:
class Cuboctahedron (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <1, 1, 0>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <1, 1, 0>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(o =  O,
                             p =  P,
                             q =  Q,
                             r =  R,
                             s =  S,
                             t =  T,
                             u =  U,
                             v =  V,
                             w =  W,
                             x =  X,
                             y =  Y,
                             z =  Z)
        self.name = "Cuboctahedron"
        
        # 6 faces
        self.faces = (('o','w','s','z'),('z','p','y','t'),
                      ('t','v','u','s'),('w','q','x','u'),
                      ('o','p','r','q'),('r','y','v','x'),
                      ('z','s','t'),('t','y','v'),
                      ('y','p','r'),('r','q','x'),
                      ('x','u','v'),('u','s','w'),
                      ('w','q','o'),('o','z','p'))

        self.edges = self._distill()

In [15]:
with open("cubocta.pov", "w") as target:
    target.write(pov_header) 
    cubocta = Cuboctahedron()
    draw_poly(cubocta, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)

![ ](cubocta.png)

In [19]:
with open("sphere_pack.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    cubocta.vert_radius = Qvector((2,1,1,0)).length()/2 # grow the balls
    cubocta.vert_color = "rgb <1,0,0>"
    draw_poly(cubocta, target, v=True, e=False, f=False)
    draw_vert(Qvector((0,0,0,0)), c=cubocta.vert_color, 
              r=cubocta.vert_radius, t=target)

In [ ]:
# !/usr/local/bin/povray +A +H768 +W1024 sphere_pack.pov

![ ](sphere_pack.png)

In [ ]:
sm_radius = (A-F).length()/2

In [ ]:
with open("smaller_sphere_pack.pov", "w") as target:
    target.write(pov_header) 
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    cub.vert_radius = sm_radius
    cub.vert_color = "rgb <1,0,0>"
    draw_poly(cub, target, v=True, e=False, f=False)
    draw_vert(Qvector((0,0,0,0)), c=cub.vert_color, 
              r=cub.vert_radius, t=target)
    

![ ](smaller_sphere_pack.png)

In [ ]:
class Icosahedron (Polyhedron):

    def __init__(self):
        # 8 vertices
        self.edge_color = "rgb <0, 1, 1>"
        self.edge_radius= 0.03
        self.vert_color = "rgb <0, 1, 1>"
        self.vert_radius= 0.03
        self.face_color = "rgb <0, 0, 0>"

        self.vertexes = dict(o =  Oi,
                             p =  Pi,
                             q =  Qi,
                             r =  Ri,
                             s =  Si,
                             t =  Ti,
                             u =  Ui,
                             v =  Vi,
                             w =  Wi,
                             x =  Xi,
                             y =  Yi,
                             z =  Zi)
        self.name = "Icosahedron"
        
        # 20 faces
        # OPPOSITE DIAGONALS of cubocta
        # ZY WX
        # RV OS
        # TU PQ
        self.faces = (('o','w','s'),('o','z','s'),
                      ('z','p','y'),('z','t','y'),
                      ('t','v','u'),('t','s','u'),
                      ('w','q','x'),('w','u','x'),
                      ('p','o','q'),('p','r','q'),
                      ('r','y','v'),('r','x','v'),
                      ('z','s','t'),('t','y','v'),
                      ('y','p','r'),('r','q','x'),
                      ('x','u','v'),('u','s','w'),
                      ('w','q','o'),('o','z','p'))

        self.edges = self._distill()

![cubocta](https://upload.wikimedia.org/wikipedia/commons/d/dc/Povlabels.gif)

In [ ]:
ico = Icosahedron()

In [ ]:
with open("jitterbug.pov", "w") as target:
    target.write(pov_header) 
    c   = Cube()
    ico = Icosahedron() * (1/PHI * math.sqrt(2)/2)
    draw_poly(c, target, f=False)
    draw_poly(ico, target, f=False)

![icosa](jitterbug.png)

In [ ]:
with open("flextegrity.pov", "w") as target:
    target.write(pov_header) 
    c   = Cube()
    draw_poly(c, target, f=False)
    for c in twelve_cubes():
        draw_poly(c, target, f=False)
    ico = Icosahedron() * (1/PHI * math.sqrt(2)/2)
    draw_poly(ico, target, f=False)
    for v in unique:
        newico = ico + Qvector(v)
        draw_poly(newico, target, f=False)

![](flextegrity.png)